In [1]:
import numpy as np
from instanseg import InstanSeg
import spatialdata as sd
import napari

c:\Users\KMK280\AppData\Local\miniconda3\envs\instanseg-env\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
c:\Users\KMK280\AppData\Local\miniconda3\envs\instanseg-env\Lib\site-packages\xarray_schema\__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


In [2]:
model = InstanSeg("fluorescence_nuclei_and_cells", verbosity=1)

Model fluorescence_nuclei_and_cells version 0.1.1 already downloaded in D:\instanseg\instanseg\utils\../bioimageio_models/, loading
Requesting default device: cuda


In [3]:
sd_path = r'C:\BLCA-2_Analysis_todel\cores\Core_000.zarr' 
sdata = sd.read_zarr(sd_path)
sdata

version mismatch: detected: RasterFormatV02, requested: FormatV04


c:\Users\KMK280\AppData\Local\miniconda3\envs\instanseg-env\Lib\site-packages\zarr\creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04


SpatialData object, with associated Zarr store: C:\BLCA-2_Analysis_todel\cores\Core_000.zarr
├── Images
│     ├── 'CD11C': DataTree[cyx] (1, 5696, 5568), (1, 2848, 2784), (1, 1424, 1392)
│     ├── 'CD44': DataTree[cyx] (1, 5696, 5568), (1, 2848, 2784), (1, 1424, 1392)
│     ├── 'CD45': DataTree[cyx] (1, 5696, 5568), (1, 2848, 2784), (1, 1424, 1392)
│     ├── 'DAPI': DataTree[cyx] (1, 5696, 5568), (1, 2848, 2784), (1, 1424, 1392)
│     ├── 'HES1': DataTree[cyx] (1, 5696, 5568), (1, 2848, 2784), (1, 1424, 1392)
│     ├── 'HLA1': DataTree[cyx] (1, 5696, 5568), (1, 2848, 2784), (1, 1424, 1392)
│     ├── 'NaKATPase': DataTree[cyx] (1, 5696, 5568), (1, 2848, 2784), (1, 1424, 1392)
│     ├── 'pCK26': DataTree[cyx] (1, 5696, 5568), (1, 2848, 2784), (1, 1424, 1392)
│     └── 'pS6': DataTree[cyx] (1, 5696, 5568), (1, 2848, 2784), (1, 1424, 1392)
└── Labels
      ├── 'blob': DataTree[yx] (5696, 5568), (2848, 2784), (1424, 1392)
      ├── 'cytoplasm': DataTree[yx] (5696, 5568), (2848, 2784), (1424

In [4]:
channels = ['DAPI','NaKATPase']
input_image = [np.array(sdata[ch]['scale0'].image) for ch in channels]
input_image = np.stack(input_image).squeeze()
input_image.shape

(2, 5696, 5568)

In [5]:
labeled_output, _ = model.eval_medium_image(input_image, resolve_cell_and_nucleus = True, pixel_size =0.3)
segm_arrays = [np.array(x).astype(int) for x in labeled_output[0, :, :, :]]

D:\instanseg\instanseg\utils\pytorch_utils.py:215: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\SparseCsrTensorImpl.cpp:55.)
  intersection = torch.sparse.mm(onehot1, onehot2.T).to_dense()


IndexError: index 423 is out of bounds for dimension 0 with size 423

In [ ]:
labeled_output.shape

torch.Size([1, 2, 5696, 5568])

In [ ]:
viewer = napari.Viewer()
viewer.add_labels(segm_arrays[1])
viewer.add_labels(segm_arrays[0])

<Labels layer 'Labels [1]' at 0x1c59e485b50>

In [34]:
segm_arrays[0]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(5696, 5568))

In [47]:
np.setdiff1d(segm_arrays[1], segm_arrays[0])

array([12932, 12933, 12934, ..., 26416, 26418, 26419], shape=(1392,))